<a href="https://colab.research.google.com/github/ag-wnl/kaggle_models/blob/main/housing_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd

In [7]:
df = pd.read_csv('https://raw.githubusercontent.com/shiv1709/House_price_prediction/master/USA_Housing.csv')
df.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address
0,79545.458574,5.682861,7.009188,4.09,23086.800503,1.059034e+06,"208 Michael Ferry Apt. 674\nLaurabury, NE 3701..."
1,79248.642455,6.002900,6.730821,3.09,40173.072174,1.505891e+06,"188 Johnson Views Suite 079\nLake Kathleen, CA..."
2,61287.067179,5.865890,8.512727,5.13,36882.159400,1.058988e+06,"9127 Elizabeth Stravenue\nDanieltown, WI 06482..."
3,63345.240046,7.188236,5.586729,3.26,34310.242831,1.260617e+06,USS Barnett\nFPO AP 44820
4,59982.197226,5.040555,7.839388,4.23,26354.109472,6.309435e+05,USNS Raymond\nFPO AE 09386


# **Exploring Dataset**

In [8]:
print(df.columns.values)

['Avg. Area Income' 'Avg. Area House Age' 'Avg. Area Number of Rooms'
 'Avg. Area Number of Bedrooms' 'Area Population' 'Price' 'Address']


In [10]:
df.shape

(5000, 7)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Avg. Area Income              5000 non-null   float64
 1   Avg. Area House Age           5000 non-null   float64
 2   Avg. Area Number of Rooms     5000 non-null   float64
 3   Avg. Area Number of Bedrooms  5000 non-null   float64
 4   Area Population               5000 non-null   float64
 5   Price                         5000 non-null   float64
 6   Address                       5000 non-null   object 
dtypes: float64(6), object(1)
memory usage: 273.6+ KB


In [14]:
for i in df.columns:
    print(f"column : {i} and no. of unique values: {len(df[i].unique())}")

#lower number of unique values => categorical, if large no => continuous values

column : Avg. Area Income and no. of unique values: 5000
column : Avg. Area House Age and no. of unique values: 5000
column : Avg. Area Number of Rooms and no. of unique values: 5000
column : Avg. Area Number of Bedrooms and no. of unique values: 255
column : Area Population and no. of unique values: 5000
column : Price and no. of unique values: 5000
column : Address and no. of unique values: 5000


In [19]:
for itr, i in enumerate(df['Address'][0:5]):
    print('\n', itr+1, ' ',i)


 1   208 Michael Ferry Apt. 674
Laurabury, NE 37010-5101

 2   188 Johnson Views Suite 079
Lake Kathleen, CA 48958

 3   9127 Elizabeth Stravenue
Danieltown, WI 06482-3489

 4   USS Barnett
FPO AP 44820

 5   USNS Raymond
FPO AE 09386


below:
1)pd.Series(i) to convert the string i into a pandas Series so that you can use the .str methods.

2)used [0] and .iloc[0] to get the first element of each extracted Series since .extract returns a DataFrame, but you are looking for individual strings.

In [28]:
for itr, i in enumerate(df['Address'][0:10]):
    print('\n')
    apart = pd.Series(i).str.extract(r'(\w+\s?\w*\.\s?\w+)')[0]  # Extracts 'Michael Ferry Apt.'
    place = pd.Series(i).str.extract(r'([A-Z][a-z]+)')[0]  # Extracts 'Laurabury'
    state = pd.Series(i).str.extract(r'([A-Z]{2})')[0]  # Extracts 'NE'
    print(f"{itr}) Apartment: {apart.iloc[0]}, Place: {place.iloc[0]}, State: {state.iloc[0]}")




0) Apartment: Ferry Apt. 674, Place: Michael, State: NE


1) Apartment: nan, Place: Johnson, State: CA


2) Apartment: nan, Place: Elizabeth, State: WI


3) Apartment: nan, Place: Barnett, State: US


4) Apartment: nan, Place: Raymond, State: US


5) Apartment: Islands Apt. 443, Place: Jennifer, State: KS


6) Apartment: nan, Place: Daniel, State: CO


7) Apartment: nan, Place: Joyce, State: TN


8) Apartment: nan, Place: Gilbert, State: US


9) Apartment: nan, Place: Unit, State: DP


##Adding Columns place and state to make more unique features hopefully

In [37]:
df['Place'] = df['Address'].str.extract(r'([A-Z][a-z]+)', expand=False)
df['State'] = df['Address'].str.extract(r'([A-Z]{2})', expand=False)

df.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address,Place,State
0,79545.458574,5.682861,7.009188,4.09,23086.800503,1.059034e+06,"208 Michael Ferry Apt. 674\nLaurabury, NE 3701...",Michael,NE
1,79248.642455,6.002900,6.730821,3.09,40173.072174,1.505891e+06,"188 Johnson Views Suite 079\nLake Kathleen, CA...",Johnson,CA
2,61287.067179,5.865890,8.512727,5.13,36882.159400,1.058988e+06,"9127 Elizabeth Stravenue\nDanieltown, WI 06482...",Elizabeth,WI
3,63345.240046,7.188236,5.586729,3.26,34310.242831,1.260617e+06,USS Barnett\nFPO AP 44820,Barnett,US
4,59982.197226,5.040555,7.839388,4.23,26354.109472,6.309435e+05,USNS Raymond\nFPO AE 09386,Raymond,US


Rounding the value of number of rooms

In [39]:
df['Avg. Area Number of Rooms'] = df['Avg. Area Number of Rooms'].round()
df['Avg. Area Number of Bedrooms'] = df['Avg. Area Number of Bedrooms'].round()
df['Avg. Area House Age'] = df['Avg. Area House Age'].round()
df.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address,Place,State
0,79545.458574,6.0,7.0,4.0,23086.800503,1.059034e+06,"208 Michael Ferry Apt. 674\nLaurabury, NE 3701...",Michael,NE
1,79248.642455,6.0,7.0,3.0,40173.072174,1.505891e+06,"188 Johnson Views Suite 079\nLake Kathleen, CA...",Johnson,CA
2,61287.067179,6.0,9.0,5.0,36882.159400,1.058988e+06,"9127 Elizabeth Stravenue\nDanieltown, WI 06482...",Elizabeth,WI
3,63345.240046,7.0,6.0,3.0,34310.242831,1.260617e+06,USS Barnett\nFPO AP 44820,Barnett,US
4,59982.197226,5.0,8.0,4.0,26354.109472,6.309435e+05,USNS Raymond\nFPO AE 09386,Raymond,US


In [40]:
for i in df.columns:
    print(f"column : {i} and no. of unique values: {len(df[i].unique())}")

column : Avg. Area Income and no. of unique values: 5000
column : Avg. Area House Age and no. of unique values: 8
column : Avg. Area Number of Rooms and no. of unique values: 9
column : Avg. Area Number of Bedrooms and no. of unique values: 5
column : Area Population and no. of unique values: 5000
column : Price and no. of unique values: 5000
column : Address and no. of unique values: 5000
column : Place and no. of unique values: 1193
column : State and no. of unique values: 62
